# Part A

# Group Members:
Jackson Ross, Brian Smith-Eitches, Kelly Zhang, Sindhu Patnam, Sam Bell

In [0]:
# Use this code if using Colab to run Selenium

# Make sure to go to Runtime -> Change runtime type and select Python 3 as runtime type and GPU as hardware accelerator

# !kill -9 -1 # Use this line to delete this VM and start a new one. 
# The above line deletes all files and folders from the current VM and allocates a new one.
!apt-get update
!pip install beautifulsoup4
!pip install selenium
# !apt-get -q update # to update ubuntu to correctly run apt install
!apt install -yq chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup, Tag, NavigableString
from selenium.webdriver.chrome.options import Options
import csv
import random
import time
import pandas as pd

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,718 kB]
Get:14 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,62

In [0]:
class SeleniumScraper(object):
    pass

    def __init__(self, number_reviews_scrape):
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        self.driver = webdriver.Chrome(options=chrome_options)
        self.delay = 5
        self.reviews = []
        self.beers = []
        self.number_reviews_scrape = number_reviews_scrape

    def go_to_page(self, url, element_type=None, element_value=None):
        try:
            self.driver.get(url)
            if element_type is not None or element_value is not None:
              if element_type == 'id':
                  self.wait(element_id=element_value)
              elif element_type == 'name':
                  self.wait(element_name=element_value)
        except TimeoutException:
            print('Loading took too long')
        except NoSuchElementException:
            print(f"Couldn\'t find {element_value} D:")
        except Exception as e:
            print(str(e))


    def wait(self, element_id=None, element_class_name=None, element_name=None):
        delay = random.randint(1, 6)
        wait = WebDriverWait(self.driver, delay)
        if element_id is not None:
            wait.until(EC.presence_of_element_located((By.ID, element_id)))
        if element_class_name is not None:
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, element_class_name)))
        if element_name is not None:
            wait.until(EC.presence_of_element_located((By.NAME, element_name)))
        else:
            time.sleep(delay)

    def get_list_of_beers(self):
        beers = self.driver.find_element_by_id('ba-content').find_element_by_tag_name('table').find_elements_by_tag_name('tr')
        self.beers = beers[1:len(beers)]  # removes the first <tr> (doesn't contain beer)

    def iterate_through_beers(self):
        init_count = len(self.beers)
        for i in range(1, init_count):
            if len(self.reviews) < self.number_reviews_scrape:
                # get page source to prevent stale elements error
                beers = self.driver.find_element_by_id('ba-content').find_element_by_tag_name('table').find_elements_by_tag_name('tr')
                beer = beers[i]
                beer.find_elements_by_tag_name('td')[1].find_element_by_tag_name('a').click()
                self.scrape_page_reviews()
                self.driver.execute_script("window.history.go(-1)")
                self.wait()

    def scrape_page_reviews(self):
        html = self.driver.page_source
        soup = BeautifulSoup(html, "lxml")
        product_name = soup.find('div', {'class': 'titleBar'})#.h1.text
        for span in product_name.findAll('span'):
            span.decompose()
        product_name = soup.find('div', {'class': 'titleBar'}).h1.text
        for user_review in soup.find_all('div', {'id': 'rating_fullview_content_2'}):
            user_score = user_review.find('span', {'class': 'BAscore_norm'}).text
            out_of_score = user_review.find('span', {'class': 'rAvg_norm'}).text.split('/')[1]

            for thing in user_review.find_all(['br', 'i', 'span', 'div']):
                thing.decompose()

            review_text = user_review.text
            #review_text = self.extract_review(user_review.find_all('br'))
            if 'rDev' in review_text[0:10]:
                review_text = review_text[6:]
                
            review_text = review_text.strip()
            
            if '=' in review_text[0:1]:
                review_text = review_text[1:]
                
            review_text = self.deEmojify(review_text)
            review_text = review_text.strip('\n')
            review_text = review_text.replace('\n', ' ')
            #print(f"{product_name} {user_score} {out_of_score} {review_text}")
            self.reviews.append({'product name': product_name, 'product review': review_text, 'user rating': user_score})
            if len(self.reviews) % 100 == 0:
              print(f'have collected {len(self.reviews)} reviews')

    def deEmojify(self, inputString):
        return inputString.encode('ascii', 'ignore').decode('ascii')

    def extract_review(self, contents):
        """
        Doesn't work when there are multiple lines of comments :(
        """
        # https://stackoverflow.com/a/5275918/7858114
        for br in contents:
            next_s = br.nextSibling
            if not (next_s and isinstance(next_s, NavigableString)):
                continue
            next2_s = next_s.nextSibling
            if next2_s and isinstance(next2_s, Tag) and next2_s.name == 'br':
                text = str(next_s).strip()
                if text:
                    return text

    def create_pandas_csv(self, filename='beerviews.csv'):
        df = pd.DataFrame(self.reviews)
        df.to_csv(filename, encoding='utf-8', index=False)

    def quit_driver(self):
        self.driver.quit()

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
url = 'https://www.beeradvocate.com/beer/top-rated/'
number_reviews_scrape = 5000
try:  
  scraper = SeleniumCraigslistScraper(number_reviews_scrape=number_reviews_scrape)
  scraper.go_to_page(url, 'id', 'ba-content')
  scraper.get_list_of_beers()
  scraper.iterate_through_beers()
  scraper.quit_driver()
  scraper.create_pandas_csv('beerreviews.csv')
except Exception as e:
  print(str(e))
  scraper.quit_driver()

have collected 100 reviews
have collected 200 reviews
have collected 300 reviews
have collected 400 reviews


# Part B

In [0]:
import pandas as pd
beerdf = pd.read_csv('beerreviews.csv')
beerdf.head(1)

,product name,product review,user rating
0,Kentucky Brunch Brand Stout,"2016 Silver Wax. Aroma has whiskey, maple, tof...",4.8


In [0]:
from string import punctuation

def removepunctuation(text):
    for itr in punctuation:
        text = text.replace(itr,'')
    return text.lower()

beerdf['clean review'] = beerdf['product review'].apply(removepunctuation)

In [0]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords # Import the stop word list
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# divides string into lists of substrings
beerdf['words'] = beerdf['clean review'].apply(word_tokenize)

def remove_stopwords(text):
  """
  Filters out words that are considered stop words
  """
  return [w for w in text if not w in stop_words] 

def remove_duplicate_words(text):
  return list(set(text))

# remove stopwords from reviews
beerdf['final words'] =  beerdf['words'].apply(remove_stopwords)
# ['the', 'beer', 'pours', 'pitch', 'black', 'with', 'a', 'frothy', 'tan', 'head', 'the', 'bottle', 'gushed', 'when', 'i', 'opened', 'it', 'it', 'smells', 'and', 'tastes', 'of', 'maple', 'syrup', 'coffee', 'creams', 'brle', 'vanilla', 'and', 'warm', 'bourbon', 'the', 'feel', 'is', 'slick', 'and', 'smooth']
# ['beer', 'pours', 'pitch', 'black', 'frothy', 'tan', 'head', 'bottle', 'gushed', 'opened', 'smells', 'tastes', 'maple', 'syrup', 'coffee', 'creams', 'brle', 'vanilla', 'warm', 'bourbon', 'feel', 'slick', 'smooth']

# some duplicates showing up in final words that would mess up similarity scores
beerdf['final words'] = beerdf['final words'].apply(remove_duplicate_words)

# getting list of words
word_count = []
for pl in range(len(beerdf)):
  word_count += beerdf['final words'][pl]

# Get frequency of words
word_freq = FreqDist(word_count)


In [0]:
# checking to see which of the attributes are used and how much
word_freq_dict = dict(word_freq)
attributes = ['aggressive', 'balanced', 'complex', 'crisp', 'fruity', 'robust']
for a in attributes:
  if word_freq_dict.get(a) is not None:
    print(f'{a} was mentioned {word_freq_dict[a]} times')

aggressive was mentioned 62 times
balanced was mentioned 611 times
complex was mentioned 442 times
crisp was mentioned 212 times
fruity was mentioned 334 times
robust was mentioned 85 times


In [0]:
#chosen_attributes = ['balanced', 'complex', 'fruity']
chosen_attributes = pd.read_excel('beerattributes.xlsx', header=None, names=['attributes'])
chosen_attr_str = ' '.join(chosen_attributes['attributes'])
print(chosen_attr_str)

balanced complex fruity


# Part C

In [0]:
 !python -m spacy download en_core_web_lg
# this is necessary to get more accurate scorings
# restart the runtime after running above line so spacy can find the package
# this takes a few minutes to download and install. It'll pop up a text input box, but you can ignore it.

     |████████████████████████████████| 826.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.1.0-cp36-none-any.whl size=828255076 sha256=e7c40e80bb5d2f478f402066f235cb8dfdc45c27fc727f872befb0ec0872cd09
  Stored in directory: /tmp/pip-ephem-wheel-cache-zyzxl5mg/wheels/b4/d7/70/426d313a459f82ed5e06cc36a50e2bb2f0ec5cb31d8e0bdf09
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [0]:
attributes_nlp = nlp(chosen_attr_str)

def get_similarity(review_words):
  text_str = ' '.join(review_words)
  review_nlp = nlp(text_str)
  return review_nlp.similarity(attributes_nlp)

In [0]:
%%time
beerdf['similarity'] =  beerdf['final words'].apply(get_similarity)

CPU times: user 1min 27s, sys: 245 ms, total: 1min 27s
Wall time: 1min 27s


In [0]:
beerdf.head(1)

,product name,product review,user rating,clean review,words,final words,similarity
0,Kentucky Brunch Brand Stout,"2016 Silver Wax. Aroma has whiskey, maple, tof...",4.8,2016 silver wax aroma has whiskey maple toffee...,"[2016, silver, wax, aroma, has, whiskey, maple...","[2016, wood, wax, coffee, aftertaste, barrel, ...",0.627137


In [0]:
beerdf['attributes'] = chosen_attr_str
beerdf.sort_values(by=['similarity'], ascending=False, inplace=True)
beerdf_top300 = beerdf[:300]

In [0]:
# beerdf_top300.to_csv('beer_attribute_similarities.csv', columns=['product name', 'clean review', 'final words', 'attributes', 'similarity'], index=False, encoding='utf-8')

In [0]:
print(beerdf_top300.shape)
print(beerdf_top300.columns)

(300, 8)
Index(['product name', 'product review', 'user rating', 'clean review',
       'words', 'final words', 'similarity', 'attributes'],
      dtype='object')


# Part D

In [0]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 2.7MB/s 


In [0]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

def sentiment_scores(sentence): 
    sid_obj = SentimentIntensityAnalyzer() 
    sentiment_dict = sid_obj.polarity_scores(sentence) 
    return sentiment_dict

In [0]:
# index 1 = product rewview
# index 2 = clean review
# index 4 = words list (stopwords included)
sentiments = []
for i in range(len(beerdf_top300)):
  sent_score = sentiment_scores(beerdf_top300.iloc[i, 1])
  sentiments.append(sent_score['compound'])
beerdf_top300['sentiment'] = sentiments
beerdf_top300.sort_values(by=['sentiment'], ascending=False, inplace=True)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
beerdf_top300.head()

,product name,product review,user rating,clean review,words,final words,similarity,attributes,sentiment
4406,Oude Geuze Cuvée Armand & Gaston,Currently #184 in the Top 250 beers list and t...,4.50,currently 184 in the top 250 beers list and th...,"[currently, 184, in, the, top, 250, beers, lis...","[high, abundance, pear, definitely, carbonated...",0.749265,balanced complex fruity,0.9978
3504,Darkness,"750 ml bottle into snifter, bottled on 9/25/20...",4.36,750 ml bottle into snifter bottled on 9252019 ...,"[750, ml, bottle, into, snifter, bottled, on, ...","[rocky, definitely, retention, biscuit, cap, b...",0.749943,balanced complex fruity,0.9976
1631,Oude Geuze Vintage,"750ml bottle, 2008 vintage, poured into a Drie...",4.58,750ml bottle 2008 vintage poured into a drie f...,"[750ml, bottle, 2008, vintage, poured, into, a...","[character, lace, retention, nuances, funk, ci...",0.755819,balanced complex fruity,0.9972
4951,Ghost In The Machine,"12 ounce bottle into tulip glass, bottled on 8...",4.28,12 ounce bottle into tulip glass bottled on 82...,"[12, ounce, bottle, into, tulip, glass, bottle...","[rocky, mildly, pear, retention, nedipa, biscu...",0.764006,balanced complex fruity,0.9966
2448,Beyond Good And Evil,"375ml bottle, 2018 release, poured into a Hill...",4.22,375ml bottle 2018 release poured into a hill f...,"[375ml, bottle, 2018, release, poured, into, a...","[lots, sweet, retention, maple, bourbon, linge...",0.740869,balanced complex fruity,0.9961


# Part E

In [0]:
beerdf_recommend = beerdf_top300[['product name', 'sentiment', 'similarity']]

In [0]:
beerdf_recommend.sort_values(by=['product name'], ascending=True, inplace=True) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
beerdf_recommend.head(100)

,product name,sentiment,similarity
3538,3rd Anniversary Imperial IPA,0.9401,0.736674
2379,A Deal With The Devil,0.9799,0.745410
1359,Aaron,0.9312,0.753289
1370,Aaron,0.9583,0.746719
536,Abner,0.9303,0.811075
...,...,...,...
1106,Double Galaxy,-0.0377,0.751708
4051,Double Nelson,0.6642,0.745429
2834,Double Shot,0.9778,0.737933
518,Double Sunshine,0.9285,0.744991


In [0]:
beerdf_recommend['sentiment_avg'] = beerdf_recommend.groupby(['product name']).sentiment.transform('mean')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
beerdf_recommend['similarity_avg'] = beerdf_recommend.groupby(['product name']).similarity.transform('mean')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
beerdf_final = beerdf_recommend.drop_duplicates(subset='product name', keep='first')

In [0]:
beerdf_other = beerdf_recommend.drop_duplicates(subset='product name', keep='first')
beerdf_other2 = beerdf_other.loc[beerdf_other['product name'].isin(["Kentucky Brunch Brand Stout", "Chemtrailmix", "Barrel-Aged Abraxas"])]

In [0]:
beerdf_other2

,product name,sentiment,similarity,sentiment_avg,similarity_avg
76,Barrel-Aged Abraxas,0.8979,0.74282,0.93245,0.749086


In [0]:
beerdf_other.head()

,product name,sentiment,similarity,sentiment_avg,similarity_avg
3538,3rd Anniversary Imperial IPA,0.9401,0.736674,0.940100,0.736674
2379,A Deal With The Devil,0.9799,0.745410,0.979900,0.745410
1359,Aaron,0.9312,0.753289,0.944750,0.750004
536,Abner,0.9303,0.811075,0.906675,0.766518
3229,Abrasive Ale,0.9904,0.759005,0.920625,0.753117


In [0]:
beerdf_final.where(beerdf_final['sentiment_avg']>0.8).sort_values(by=['similarity_avg', 'sentiment_avg'], ascending=False).head(10)

,product name,sentiment,similarity,sentiment_avg,similarity_avg
2063,Galaxy Dry Hopped Fort Point Pale Ale,0.9396,0.787919,0.939600,0.787919
1254,Framboise Du Fermier,0.9118,0.776859,0.911800,0.776859
1154,Cable Car,0.7906,0.774401,0.862350,0.772645
1450,Nectarine Premiere,0.8513,0.772397,0.851300,0.772397
2038,Flora Plum,0.8600,0.772479,0.871450,0.768436
536,Abner,0.9303,0.811075,0.906675,0.766518
2979,Pseudo Sue,0.9286,0.781681,0.951200,0.765275
2891,Great,0.9867,0.764486,0.986700,0.764486
4872,Black & Wild,0.9871,0.763929,0.987100,0.763929
4389,Crusher,0.8173,0.749887,0.900167,0.763678


**ANALYSIS:** Based on the reviews we analyzed, we would suggest Galaxy Dry Hopped Fort Point Pale Ale, Framboise Du Fermier, and Cable Car for someone who likes balanced, complex, and fruity beers. These three beers were the three highest similarity beers to the desired attributes with sentiment scores above .80. We added the last caveat because there were some beers that were high in similarity but had low sentiment scores.

# Part F

In [0]:
# beerdf.to_csv('beer_rating.csv', columns=['product name', 'user rating'], index=False, encoding='utf-8')

In [0]:
beerdf_userrating = pd.read_csv('beerreviews.csv')

In [0]:
beerdf_userrating = beerdf_userrating[['product name', 'user rating']].rename(columns={"product name": "productname", "user rating": "userrating"})

In [0]:
beerdf_userrating.head()

,productname,userrating
0,Kentucky Brunch Brand Stout,4.80
1,Kentucky Brunch Brand Stout,4.74
2,Kentucky Brunch Brand Stout,4.68
3,Kentucky Brunch Brand Stout,5.00
4,Kentucky Brunch Brand Stout,4.97


In [0]:
beerdf_userrating['user_rating_avg'] = beerdf_userrating.groupby(['productname']).userrating.transform('mean')

In [0]:
beerdf_userrating.head()

,productname,userrating,user_rating_avg
0,Kentucky Brunch Brand Stout,4.80,4.812
1,Kentucky Brunch Brand Stout,4.74,4.812
2,Kentucky Brunch Brand Stout,4.68,4.812
3,Kentucky Brunch Brand Stout,5.00,4.812
4,Kentucky Brunch Brand Stout,4.97,4.812


In [0]:
beerdf_userrating_final = beerdf_userrating.drop_duplicates(subset='productname', keep='first').sort_values(by=['user_rating_avg'], ascending=False).head(10)

In [0]:
beerdf_userrating_final

,productname,userrating,user_rating_avg
0,Kentucky Brunch Brand Stout,4.80,4.812000
1747,Chemtrailmix,5.00,4.811176
75,Barrel-Aged Abraxas,4.74,4.798000
225,King JJJuliusss,5.00,4.789600
2459,It Was All A Dream,4.66,4.785000
1300,Very GGGreennn,4.69,4.776818
475,Zenne Y Frontera,5.00,4.764000
675,Barrel Aged Imperial German Chocolate Cupcake ...,4.75,4.755600
275,SR-71,4.84,4.746400
1914,Anabasis,4.66,4.726000


**ANALYSIS:** Based only on average rating, we would recommend Kentucky Brunch Brand Stout, Chemtrailmix, and Barrel-Aged Abraxas. Of those three, only one of them is even in the top 300 of similarity with the user's requirements, Barrel-Aged Abraxas. This beer has an average sentiment of 0.93245	and an average similarity of 0.749086. The others have similarity that we consider too low to meet the requirements. In fact, none of the top 10 in the entire dataset match any of the top 10 from part E.